In [1]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train JAX/Flax model locally and use `jax2tf` to convert to SavedModel

In [2]:
import logging
import os

import tensorflow as tf
import tensorflow_datasets as tfds
from absl import flags
from jax.experimental.jax2tf.examples import mnist_lib
from jax.experimental.jax2tf.examples import saved_model_lib

In [3]:
TRAIN_BATCH_SIZE = 128
TEST_BATCH_SIZE = 16
NUM_EPOCHS = 2

PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

REGION = "us-central1"
os.environ['REGION'] = REGION

BUCKET_NAME = PROJECT_ID
os.environ['BUCKET_NAME'] = BUCKET_NAME
# Use a regional bucket in the above region you have rights to.
# Create if needed:
# !gsutil mb -l ${REGION} gs://${BUCKET_NAME}

MODEL_BASE_PATH = f"gs://{BUCKET_NAME}/savedmodels"
MODEL_NAME = "jax_model_local"
MODEL_VERSION = 1

# Block TF from the GPU to let JAX use it all
tf.config.set_visible_devices([], 'GPU')

logger = logging.getLogger()

# need to initialize flags somehow to avoid errors in load_mnist
flags.FLAGS([''])

flax_mnist = mnist_lib.FlaxMNIST()

In [4]:
train_ds = mnist_lib.load_mnist(tfds.Split.TRAIN, TRAIN_BATCH_SIZE)
test_ds = mnist_lib.load_mnist(tfds.Split.TEST, TEST_BATCH_SIZE)

In [5]:
image, _ = next(iter(train_ds))
input_signature = tf.TensorSpec.from_tensor(
    tf.expand_dims(image[0], axis=0)
)

In [6]:
logger_level = logger.level
logger.setLevel(logging.INFO)
predict_fn, params = flax_mnist.train(
    train_ds=train_ds,
    test_ds=test_ds,
    num_epochs=NUM_EPOCHS,
)
logger.setLevel(logger_level)

INFO:absl:Starting the local TPU driver.
INFO:absl:Unable to initialize backend 'tpu_driver': Not found: Unable to find driver in registry given worker: local://
INFO:absl:Unable to initialize backend 'tpu': Invalid argument: TpuPlatform is not available.
INFO:root:mnist_flax: Epoch 0 in 5.47 sec
INFO:root:mnist_flax: Training set accuracy 88.41%
INFO:root:mnist_flax: Test set accuracy 88.96%
INFO:root:mnist_flax: Epoch 1 in 1.13 sec
INFO:root:mnist_flax: Training set accuracy 90.85%
INFO:root:mnist_flax: Test set accuracy 91.26%


In [7]:
saved_model_lib.convert_and_save_model(
    jax_fn=predict_fn,
    params=params,
    model_dir=os.path.join(MODEL_BASE_PATH, MODEL_NAME, "model", str(MODEL_VERSION)),
    input_signatures=[input_signature],
)

INFO:tensorflow:Assets written to: gs://dsparing-sandbox/savedmodels/jax_model_local/model/1/assets


INFO:tensorflow:Assets written to: gs://dsparing-sandbox/savedmodels/jax_model_local/model/1/assets


## Test Prediction with both predict_fn and savedmodel

In [8]:
image_to_predict, _ = next(iter(mnist_lib.load_mnist(tfds.Split.TEST, batch_size=1)))

In [9]:
predict_fn(params, image_to_predict)

DeviceArray([[ -9.100128  , -17.533741  ,  -7.954192  ,  -9.037198  ,
               -2.8668425 ,  -8.061162  ,  -9.226994  ,  -4.7232003 ,
               -6.6714783 ,  -0.07045154]], dtype=float32)

In [13]:
!gsutil ls -l $MODEL_BASE_PATH/$MODEL_NAME/model/$MODEL_VERSION

         0  2021-06-28T08:57:54Z  gs://dsparing-sandbox/savedmodels/jax_model_local/model/1/
     53991  2021-06-28T08:57:59Z  gs://dsparing-sandbox/savedmodels/jax_model_local/model/1/saved_model.pb
                                 gs://dsparing-sandbox/savedmodels/jax_model_local/model/1/assets/
                                 gs://dsparing-sandbox/savedmodels/jax_model_local/model/1/variables/
TOTAL: 2 objects, 53991 bytes (52.73 KiB)


In [14]:
!saved_model_cli show --dir $MODEL_BASE_PATH/$MODEL_NAME/model/$MODEL_VERSION --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (1, 28, 28, 1)
        name: serving_default_inputs:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['output_0'] tensor_info:
        dtype: DT_FLOAT
        shape: (1, 10)
        name: StatefulPartitionedCall:0
  Method name is: tensorflow/serving/predict

Defined Functions:
  Function Name: '__call__'
    Option #1
      Callable with:
        Argument #1
          inputs: TensorSpec(sh

In [15]:
loaded_model = tf.saved_model.load(os.path.join(MODEL_BASE_PATH, MODEL_NAME, "model", str(MODEL_VERSION)))
loaded_model.signatures["serving_default"](image_to_predict)

{'output_0': <tf.Tensor: shape=(1, 10), dtype=float32, numpy=
 array([[ -9.100126  , -17.533737  ,  -7.954193  ,  -9.0372    ,
          -2.866843  ,  -8.061165  ,  -9.226992  ,  -4.7232056 ,
          -6.6714773 ,  -0.07045154]], dtype=float32)>}